<a href="https://colab.research.google.com/github/faithNassiwa/predictive-diagnosis-assistant/blob/main/Dataset2_SVM_dropNAN_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

from pprint import pprint

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Read the training and test dataset -- Took a minute
train_df = pd.read_csv('/content/drive/MyDrive/Project/processed_train.csv', low_memory=False)
test_df = pd.read_csv('/content/drive/MyDrive/Project/processed_test.csv', low_memory=False)
pprint(f'Train dataset shape: {train_df.shape}')
pprint(f'Test dataset shape: {test_df.shape}')


'Train dataset shape: (1025602, 221)'
'Test dataset shape: (134529, 221)'


In [10]:
columns_remove = ["Have you noticed a diffuse (widespread) redness in one or both eyes?",
         "Have you lost consciousness associated with violent and sustained muscle contractions or had an absence episode?",
         "Have you had any vaginal discharge?"]

In [11]:
train_df = train_df.drop(columns=columns_remove)

In [12]:
test_df = test_df.drop(columns=columns_remove)

In [13]:
train_df = train_df.dropna()
test_df = test_df.dropna()
pprint(f'Train dataset shape: {train_df.shape}')
pprint(f'Test dataset shape: {test_df.shape}')


'Train dataset shape: (982224, 218)'
'Test dataset shape: (128726, 218)'


In [14]:
# Prepare training and test variables
X_columns = [col for col in train_df.columns if col not in ['Unnamed: 0', 'PATHOLOGY']]
X_train = train_df[X_columns]
y_train = train_df['PATHOLOGY']
X_test = test_df[X_columns]
y_test = test_df['PATHOLOGY']
pprint(f'X_train shape: {X_train.shape}')
pprint(f'y_train shape: {y_train.shape}')
pprint(f'X_test shape: {X_test.shape}')
pprint(f'y_test shape: {y_test.shape}')


'X_train shape: (982224, 216)'
'y_train shape: (982224,)'
'X_test shape: (128726, 216)'
'y_test shape: (128726,)'


In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
categorical = X_train.select_dtypes("object").columns

In [17]:
categorical

Index(['SEX', 'Characterize your pain:', 'Do you feel pain somewhere?',
       'Does the pain radiate to another location?',
       'Have you traveled out of the country in the last 4 weeks?',
       'What color is the rash?', 'Do your lesions peel off?',
       'Where is the affected region located?',
       'Is the lesion (or are the lesions) larger than 1cm?',
       'Where is the swelling located?'],
      dtype='object')

In [18]:
categorical = [col for col in X_train.columns if X_train[col].dtype == 'O']


In [19]:
label_encoder = LabelEncoder()

In [20]:
for column in categorical:
    X_train[column] = label_encoder.fit_transform(X_train[column])

<ipython-input-20-97d03cb2f126>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[column] = label_encoder.fit_transform(X_train[column])
<ipython-input-20-97d03cb2f126>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[column] = label_encoder.fit_transform(X_train[column])
<ipython-input-20-97d03cb2f126>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [21]:
for column in categorical:
    X_test[column] = label_encoder.fit_transform(X_test[column])

<ipython-input-21-d013676cc0e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[column] = label_encoder.fit_transform(X_test[column])
<ipython-input-21-d013676cc0e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[column] = label_encoder.fit_transform(X_test[column])
<ipython-input-21-d013676cc0e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [22]:
svm_classifier = SVC(kernel='linear', C=1.0)

In [23]:
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [24]:
y_pred = svm_classifier.predict(X_test)

In [5]:
from sklearn.preprocessing import LabelBinarizer

In [6]:
lb = LabelBinarizer()

In [25]:
y_pred_encoded = lb.fit_transform(y_pred)
y_test_encoded = lb.fit_transform(y_test)

In [61]:
y_test_encoded

array([21, 13,  1, ..., 37, 10, 39])

In [27]:
import sklearn.metrics as metrics

In [28]:
#calculate AUC of model
auc = metrics.roc_auc_score(y_test_encoded, y_pred_encoded)

#print AUC score
print(auc)


0.9913478669518212


In [29]:
accuracy = accuracy_score(y_test, y_pred)
print(f'The accuracy score is {accuracy * 100:.2f}%')
print("Classification Report")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

The accuracy score is 98.71%
Classification Report
                                          precision    recall  f1-score   support

     Acute COPD exacerbation / infection       1.00      1.00      1.00      2153
                Acute dystonic reactions       1.00      1.00      1.00      3302
                        Acute laryngitis       0.99      0.99      0.99      3214
                      Acute otitis media       1.00      1.00      1.00      3509
                   Acute pulmonary edema       1.00      1.00      1.00      2598
                    Acute rhinosinusitis       1.00      0.67      0.80      1829
                      Allergic sinusitis       1.00      1.00      1.00      2411
                             Anaphylaxis       1.00      1.00      1.00      3798
                                  Anemia       1.00      1.00      1.00      6770
                     Atrial fibrillation       1.00      1.00      1.00      2831
                               Boerhaave      

In [31]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [32]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)


In [33]:
X = X_train
y = y_train

In [34]:
scores = cross_val_score(svm_classifier, X, y, cv=kf, scoring='accuracy') # about 3 hours to run


In [35]:
for fold, score in enumerate(scores):
    print(f"Fold {fold+1} Score: {score:.4f}")


Fold 1 Score: 0.9940
Fold 2 Score: 0.9940
Fold 3 Score: 0.9942
Fold 4 Score: 0.9940
Fold 5 Score: 0.9940


In [36]:
mean_score = np.mean(scores)
std_score = np.std(scores)
print(f"Mean Score: {mean_score:.4f}")
print(f"Standard Deviation: {std_score:.4f}")

Mean Score: 0.9940
Standard Deviation: 0.0001


In [38]:
accuracy = accuracy_score(y_test, y_pred)
print(f'The accuracy score is {accuracy * 100:.2f}%')
print("Classification Report")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

The accuracy score is 98.71%
Classification Report
                                          precision    recall  f1-score   support

     Acute COPD exacerbation / infection       1.00      1.00      1.00      2153
                Acute dystonic reactions       1.00      1.00      1.00      3302
                        Acute laryngitis       0.99      0.99      0.99      3214
                      Acute otitis media       1.00      1.00      1.00      3509
                   Acute pulmonary edema       1.00      1.00      1.00      2598
                    Acute rhinosinusitis       1.00      0.67      0.80      1829
                      Allergic sinusitis       1.00      1.00      1.00      2411
                             Anaphylaxis       1.00      1.00      1.00      3798
                                  Anemia       1.00      1.00      1.00      6770
                     Atrial fibrillation       1.00      1.00      1.00      2831
                               Boerhaave      